# Environment_Tests

These tests are grouped into a single notebook called `all_tests.ipynb` which you can run using the `ope test` command. Running this command outputs the results of the tests, separating them into two groups of `PASSED TESTS` and `FAILED TESTS`. We keep track of these groups using the global variables `ERRORS` and `PASSES` which are initialized at the last cell. 
We also use the helper function `runshell` to verify that any shell commands being tested do not run into any exceptions.

### Helper function for shell commands

In [ ]:
import subprocess

# function to execute shell commands (this is for convenience)
def runshell(CMD):
    global ERRORS
    try:
        result = subprocess.check_output(CMD, shell=True, stderr=subprocess.STDOUT)
        return 0, result.decode('utf-8')
    except subprocess.CalledProcessError as e:
        return e.returncode, e.output.decode('utf-8')
    except Exception as e:  # new add: handle exceptions 
        return -1, str(e)

### Write Permission to Home Directory Test

This test confirms that the notebook user has permission to write, read, and execute files within their `/home` directory

In [4]:
# Test to check write permissions to home directory
def WritePermissionTest():

    TEST = "WRITE PERMISSION TO HOME DIRECTORY"
    CMD = f"touch {os.path.expanduser('~')}/test_write_permissions.tmp && echo 'Write Permission: Yes' && rm {os.path.expanduser('~')}/test_write_permissions.tmp || echo 'Write Permission: No'"

    e, output = runshell(CMD)

    if e == 0:
        PASSES.append("Write Permission to Home Directory test")

    else:
        ERRORS.append(output)


### Environmental Variables test

This test confirms that the environment variables set from Dockerfile still exist and maintain the same values within the Jupyter Notebook. It also ensures that the user has a valid UID and GID within a customizable range

In [ ]:
#Verify environment variables are correct

def check_environment_test():
    
    NB_UID = int(os.environ['NB_UID'])
    NB_GID = int(os.environ['NB_GID'])
    NB_GROUP = os.environ['NB_GROUP']
    NB_USER = os.environ['NB_USER']

    XDG_CACHE_HOME = f"/home/{NB_USER}/.cache/"

    UID_LOWER_BOUND = 1000  
    UID_UPPER_BOUND = 1000810001 

    GID_LOWER_BOUND = 0  
    GID_UPPER_BOUND = 10000 

    EXPECTED_NB_GROUP = 'root'

    err = []    
    
    if not (UID_LOWER_BOUND <= NB_UID <= UID_UPPER_BOUND):
         err.append(f"NB_UID {NB_UID} is not within the acceptable range: {UID_LOWER_BOUND}-{UID_UPPER_BOUND}.")
         err.append("Acceptable ranges can be modified within: " + os.getcwd()) 

    if not (GID_LOWER_BOUND <= NB_GID <= GID_UPPER_BOUND):
         err.append(f"NB_GID {NB_GID} is not within the acceptable range: {GID_LOWER_BOUND}-{GID_UPPER_BOUND}.")
         err.append("Acceptable ranges can be modified within: " + os.getcwd())              
            
    if  NB_GROUP != EXPECTED_NB_GROUP:
         err.append("NB_GROUP does not match " + "'" + EXPECTED_NB_GROUP + "'")

    if NB_USER != 'jovyan':
         err.append("NB_USER does not match 'jovyan'.")

    if XDG_CACHE_HOME != '/home/jovyan/.cache/':
         err.append("XDD_CACHE_HOME does not match expected path: /home/jovyan/.cache/")

    s = '\n '.join(err)
    if not s: # no errors
        PASSES.append("Environmental Variables test")
    else:
        ERRORS.append(f"Environmental Variables test:\n {s}")
       

### ASLR Test

In certain classes, we want to disable ASLR so that students can use GDB properly in their environments. This will help us verify the status of ASLR.

In [ ]:
# run gdb 100 times on date binary stopping at first instruction and grep ld. If ld is found at same address (following 
# command has a value of 1 then ASLR is properly disabled.
def ASLRtest():
    e, count = runshell("echo $(for ((i=0;i<100; i++)); do gdb -ex starti -ex quit -q --batch /usr/bin/date 2>/dev/null | grep ld; done | tee out | uniq | wc -l)")
    if count == 1: 
        PASSES.append("ASLR test")
    else:
        e2, output2 = runshell("echo $(cat /proc/sys/kernel/randomize_va_space)")
        ERRORS.append(f"ASLR test ERROR: ASLR status is enabled with status {output2[0]} (0 is disabled, 1 is partial and 2 is full)")

### Network Test
This test ensures that the container has a connection to the internet. We use both `ping` and `curl` because some systems may not have one of these utilities by default—in our case, our cluster configuration did not include `ping` by default. If either `ping` or `curl` works properly, then the container is connected to the internet and the test passes, otherwise it fails.

In [ ]:
# Curl test to check internet connectivity
def NetworkTest():
    e, output = runshell("curl google.com")
    if e == 0:
        PASSES.append("Network test")
    else:
        ERRORS.append("Network test ERROR: " + output)

### Pip-Conda Test

Ensures that the user can install Python-based packages using `pip`

In [27]:
def PipCondaTest():
    TEST = "PIP PACKAGE INSTALLATION"
    CMD = "pip install --user pytest"
    e, output = runshell(CMD)

    if e == 0:
        PASSES.append("Pip-Conda test")
    else:
        ERRORS.append("Pip-Conda test: " + output)

### Git and SSH Test

Tests if Git and SSH configuration files are permanent and working

In [1]:
def GitSSHTest():

    e, output = runshell("readlink -f ~/.gitconfig")

    if e == 0:
        PASSES.append("Git config test")
    else:
        ERRORS.append("Git config test:" + output)

    e, output = runshell("readlink -f /etc/ssh/ssh_config")

    if e == 0:
        PASSES.append("ssh config test")

    else:
        ERRORS.append("ssh config test:" + output)

### Conda Directory Test

Test if conda directories are read/writable

In [30]:
def check_permissions():
    
    #Identify conda directories

    conda_base_dir = os.path.abspath(os.path.join(os.path.dirname(os.sys.executable), ".."))
    conda_env_dir = os.environ.get('CONDA_PREFIX', '')

    unaccessible_dirs = 0    
    
    for dir_name, dir_path in [('Conda Base Directory', conda_base_dir), ('Conda Environment Directory', conda_env_dir)]:
        try:
            readable = os.access(dir_path, os.R_OK)
            writable = os.access(dir_path, os.W_OK)
            if not (readable and writable):
                unaccessible_dirs += 1
                ERRORS.append(f"Conda Directory test ERROR: {dir_name} at '{dir_path}' is not readable/writable")
        except Exception as e:
            ERRORS.append(f"Conda Directory test ERROR: Error checking permissions for {dir_name} at {dir_path}: {e}")
            unaccessible_dirs += 1

    if unaccessible_dirs == 0:
        PASSES.append("Conda directory r/w test passed")


### Running the tests

The final cell where each test is called. `ERRORS` and `PASSES` are arrays used to contain the results of the tests.

In [ ]:
import os

# global var to keep track of test results we have
ERRORS = []
PASSES = []


#Run the tests

WritePermissionTest()
check_environment_test()
ASLRtest()
NetworkTest()
GitSSHTest()
check_permissions()

#Print the results
if len(ERRORS) > 0:
    print("FAILED TESTS:")
    print('-' + '\n-'.join(ERRORS))
    print('\n')
    print("PASSED TESTS:")
    print('-' + '\n-'.join(PASSES))
else:
    print("ALL TESTS PASS")
    print('-' + '\n-'.join(PASSES))

Additional tests can be added to suit the compatability needs of users and textbooks. They may be written in this format:

In [ ]:
# Define the test and conditions

def perform_test():

    test_command = "<INSERT TEST COMMAND HERE>"
    expected_result = "<INSERT EXPECTED RESULT HERE>"
    environment_variables = "<INSERT REQUIRED VARIABLES IF APPLICABLE>"

    execution_result, output = runshell(test_command)

    if output == expected_result:
        # If test passes, add to the PASSES array 
        PASSES.append("<TEST NAME> PASSED")
    else:
        # If test fails, add to the ERRORS array with error message
        ERRORS.append("<TEST NAME> FAILED: " + output)


Test Authors: Ross Mikulskis, Jonathan Mikalov, Yuxie Ge, Yiqin Zhang